## Testing how to filter the error with Hydroseq, while keeping dam info

In [16]:
import pandas as pd
import numpy as np
# hydroseq_dups1 = pd.read_csv('extracted_HUC1019.csv', usecols=['Hydroseq', 'UpHydroseq', 'DnHydroseq', 'LENGTHKM', 'DamID', 'WKT', 'Norm_stor'])
hydroseq_dups1 = pd.read_csv('Rio_Grande2.csv', usecols=['Hydroseq', 'UpHydroseq', 'DnHydroseq', 'LENGTHKM', 'DamID', 'WKT', 'Norm_stor'])

FileNotFoundError: [Errno 2] File b'Rio_Grande2.csv' does not exist: b'Rio_Grande2.csv'

In [17]:
# ## Original testing
# #Keep only the last Hydroseq
# hydroseq_dups = hydroseq_dups1.drop_duplicates(subset='Hydroseq', keep="last")  #drop everything but last duplicate
# test_df = hydroseq_dups.drop(columns=['Norm_stor'])  #drop Norm_stor
# #Group by hydroseq and sum the storage
# test_stor = hydroseq_dups.groupby(['Hydroseq'])['Norm_stor'].sum().reset_index()
# #Count # of duplicate dams
# hydroseq_dups1['DamCount'] = np.zeros(len(hydroseq_dups1))
# count_df = hydroseq_dups1.pivot_table(index=['Hydroseq'], aggfunc='size').reset_index()
# #Merge the dataframes so the storage and DamIDs are how we want
# merged = test_df.merge(test_stor, how= 'left', on='Hydroseq') # Merge NABD and NHD
# # m = count_df[count_df.Hydroseq==550027928]
# print(count_df)
# # print(m)
# # print(merged)

## Debugging from extract.py
nabd_nhd_join = hydroseq_dups1.copy()
 # Add stuff for bifurcation analysis
## add a column to keep track of steps
nabd_nhd_join.insert(5, "step", np.zeros(len(nabd_nhd_join)), True)
  
## Filtering the Hydroseq, storage, and dam count
# Group by Hydroseq and sum storage
storage_sum = nabd_nhd_join.groupby(['Hydroseq'])['Norm_stor'].sum().reset_index()
# print(type(storage_sum))
# print(storage_sum.columns)
  
# Count # of duplicate dams
nabd_nhd_join['DamCount'] = np.zeros(len(nabd_nhd_join))  #add count column
dam_count = nabd_nhd_join.pivot_table(index=['Hydroseq'], aggfunc={'DamCount':'size'})
dam_count = dam_count.reset_index() #Aggregate by the size of each Hydroseq
# print(type(dam_count))
# print(dam_count.columns)
    
# Merge count and storage dataframes
count_sum_merge = storage_sum.merge(dam_count, how= 'left', on='Hydroseq')  #merge count and sum 
# print(type(count_sum_merge))
# print(count_sum_merge.columns)
  
# Filter nabd_nhd_join for merge
nabd_nhd_filtered = nabd_nhd_join.drop_duplicates(subset='Hydroseq', keep="last")  #drop everything but last duplicate
nabd_nhd_filtered = nabd_nhd_filtered.drop(columns=['Norm_stor', 'DamCount'])  #drop Norm_stor and DamCount, so new one is added
# print(type(nabd_nhd_filtered))
# print(nabd_nhd_filtered.columns)
  
# Merge the dataframes so the storage and DamIDs are how we want
nabd_nhd_df = nabd_nhd_filtered.merge(count_sum_merge, how= 'left', on='Hydroseq') #Merge filtered dataframes


print('I ran')
print(nabd_nhd_df.head(5))

I ran
     DamID                                                WKT  LENGTHKM  \
0  19785.0  MULTILINESTRING ZM ((-96.6388022271039 45.7687...     2.044   
1  28196.0  MULTILINESTRING ZM ((-97.0208380931776 47.9225...     6.137   
2  19788.0  MULTILINESTRING ZM ((-96.1197142279096 45.8723...     1.072   
3  19789.0  MULTILINESTRING ZM ((-96.0003330947616 45.8709...     3.585   
4  19790.0  MULTILINESTRING ZM ((-95.5445158288025 47.0275...     0.230   

    Hydroseq  step  UpHydroseq  DnHydroseq  Norm_stor  DamCount  
0  840002063   0.0   840002093   840002034   106000.0         1  
1  840000151   0.0   840000153   840000150     5350.0         2  
2  840003868   0.0   840003962   840003772      700.0         1  
3  840028840   0.0           0   840004171      400.0         1  
4  840004526   0.0   840004650   840004401     3500.0         1  


In [4]:
Hduplicates = pd.concat(g for _, g in seq_dup.groupby("Hydroseq") if len(g) > 1)  #group duplicates together
# flowlines = flowlines.drop_duplicates(subset='Hydroseq', keep="last")  #drop everything but last duplicate
print(Hduplicates)

COMID  Norm_stor   DamID  \
87   2889298.0    11234.0  3816.0   
88   2889298.0    11234.0  3817.0   
101  2891624.0    41811.0  3832.0   
102  2891624.0    41000.0  4313.0   
146  5241412.0      130.0  3883.0   
147  5241412.0       41.0  4370.0   
71   2885282.0       30.0  3798.0   
72   2885282.0      257.0  3799.0   
189   228551.0       19.0  4264.0   
190   228551.0       38.0  4265.0   
191   228551.0       99.0  4266.0   
192   228551.0       14.3  4422.0   

                                                   WKT  LENGTHKM  REACHCODE  \
87   MULTILINESTRING ZM ((-105.182906280508 40.0301...     5.134     1019.0   
88   MULTILINESTRING ZM ((-105.182906280508 40.0301...     5.134     1019.0   
101  MULTILINESTRING ZM ((-105.358636280236 39.9485...     0.085     1019.0   
102  MULTILINESTRING ZM ((-105.358636280236 39.9485...     0.085     1019.0   
146  MULTILINESTRING ZM ((-105.396787280176 38.9059...     1.432     1019.0   
147  MULTILINESTRING ZM ((-105.396787280176 38.9059..

In [7]:
import numpy as np
import geopandas as gp
import pandas as pd
from pathlib import Path
gdrive = Path("/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/GIS/Layers") #where shapefiles/csv live 

nabd = pd.DataFrame()
def extract_dams(nabd):
    ## NABD
    nabd = gp.read_file(gdrive/"nabd_fish_barriers_2012.shp")  #read in NABD from Drive
    nabd = nabd.drop_duplicates(subset='NIDID', keep="first")  #drop everything after first duplicate
    nabd["DamID"] = range(len(nabd.COMID))  #add DamID 
    # print(nabd.DamID.unique)  #check the DamIDs
    return nabd

In [8]:
print(extract_dams(nabd))

COMMENT    NIDID        COMID UNIQUE_STR        newX       newY  \
0           1  AL00288     893441.0          1  -86.196373  31.424403   
1           1  AL01925     894119.0          2  -86.400374  31.170262   
2           1  AL00648     895019.0          3  -86.299755  31.223052   
3           1  AL00652     895035.0          4  -86.450075  31.179562   
4           1  AL00641     895041.0          5  -86.346634  31.143277   
...       ...      ...          ...        ...         ...        ...   
52451       1  AK00189   41700279.0      52561 -149.551141  61.348327   
52452       1  AK00029   41701413.0      52562 -149.924029  61.207976   
52453       1  AK00060   62834739.0      52563 -149.457555  60.102455   
52454       1  AK00019   62833387.0      52564 -148.077985  60.048443   
52455       1  AK00207  125387387.0      52565 -152.455619  57.920862   

       RecordID              Dam_name Dam_former     STATEID  ... Num_locks  \
0         326.0    DONALDSON LAKE DAM       None  

In [5]:
def my_function():
  print("Hello from a very nice function")


In [6]:
print(my_function())

Hello from a very nice function
None


In [21]:
coast_huc = pd.read_csv('extracted_HUC0109.csv', usecols=['Hydroseq', 'REACHCODE', 'FTYPE', 'FCODE'])
print(coast_huc.head(3))
print(coast_huc.FCODE.unique())
print(coast_huc.FTYPE.unique())
print(len(coast_huc))

Hydroseq  REACHCODE           FTYPE  FCODE
0  150034392      109.0  ArtificialPath  55800
1  150023613      109.0     StreamRiver  46006
2  150057223      109.0  ArtificialPath  55800
[55800 46006 46003 33400 56600 33600]
['ArtificialPath' 'StreamRiver' 'Connector' 'Coastline' 'CanalDitch']
6786


In [22]:
#Filtering
coast_huc = coast_huc[coast_huc['FCODE']!= 56600]
print(coast_huc)
print(len(coast_huc))

Hydroseq  REACHCODE           FTYPE  FCODE
0     150034392      109.0  ArtificialPath  55800
1     150023613      109.0     StreamRiver  46006
2     150057223      109.0  ArtificialPath  55800
3     150021854      109.0     StreamRiver  46006
4     150043891      109.0     StreamRiver  46006
...         ...        ...             ...    ...
6778  150015301      109.0     StreamRiver  46006
6779  150014599      109.0     StreamRiver  46006
6780  150031769      109.0     StreamRiver  46006
6781  150037921      109.0     StreamRiver  46006
6782  150048935      109.0     StreamRiver  46006

[5827 rows x 4 columns]
5827
